In [1]:
import tensorflow as tf
from IPython.display import clear_output
tf.keras.utils.set_random_seed(42)

### Port fashion mnist dataset with (un)known intrinsic dimension:

In [2]:
fashion_mnist = tf.keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

train_images = train_images / 255.0

test_images = test_images / 255.0

In [3]:
data = train_images
data = data.reshape((len(data), -1))

### Investigate loss on different types of autoencoders (and mixes of IDs)

In [4]:
X_train = train_images
X_valid = test_images

In [5]:
def coefficient(low, id, high):
    return (high-id)/(high-low)

In [6]:
ID = 12
radius = 7

input = tf.keras.layers.Input(shape=(28,28))

flatten = tf.keras.layers.Flatten(input_shape=(28, 28))(input)

latent_minus = tf.keras.layers.Dense(max(ID-radius, 1), activation="relu")(flatten)
latent_id = tf.keras.layers.Dense(ID, activation="relu")(flatten)
latent_plus = tf.keras.layers.Dense(ID+radius, activation="relu")(flatten)

hidden_minus = tf.keras.layers.Dense(28 * 28)(latent_minus)
hidden_id = tf.keras.layers.Dense(28 * 28)(latent_id)
hidden_plus = tf.keras.layers.Dense(28 * 28)(latent_plus)

output_minus = tf.keras.layers.Reshape([28, 28])(hidden_minus)
output_id = tf.keras.layers.Reshape([28, 28])(hidden_id)
output_plus = tf.keras.layers.Reshape([28, 28])(hidden_plus)

model = tf.keras.Model(inputs=[input],
                       outputs=[output_minus, output_id, output_plus])

model.compile(loss="mse", optimizer="nadam",
              metrics=["RootMeanSquaredError"])
history = model.fit(X_train, X_train, epochs=10,
                         validation_data=(X_valid, X_valid))

results = []

for i in history.history:
    results.append((i, history.history[i][-1]))

# 'ID - radius' validation loss
(ID-radius), results[8][1]

clear_output(wait=True)

print("Result of {}: {}\n".format(max(ID-radius, 1),results[8][1]))
print("Result of {}: {}\n".format(ID,results[9][1]))
print("Result of {}: {}\n".format(ID+radius,results[10][1]))

c = coefficient(results[8][1], results[9][1], results[10][1])

print("Coefficient: {}\n".format(c))
print("Elbow Calculation: {}\n".format(c/results[9][1]))

Result of 5: 0.0334358811378479

Result of 12: 0.023712586611509323

Result of 19: 0.01936451904475689

Coefficient: 0.3090011853854091

Elbow Calculation: 13.031104132496028

